## Prefect 2 Tutorial - Intro

In [1]:
from prefect import flow, task
import requests
import asyncio

In [2]:
!prefect version

Version:             2.0.0
API version:         0.8.0
Python version:      3.9.12
Git commit:          29889eb4
Built:               Wed, Jul 27, 2022 12:44 PM
OS/Arch:             darwin/arm64
Profile:             default
Server type:         ephemeral
Server:
  Database:          sqlite
  SQLite version:    3.37.0


### Run a basic flow

The simplest way to begin with Prefect is to import flow and annotate your a Python function using the `@flow` decorator:

In [3]:
@task 
def add_one(x):
    return x + 1

@flow 
def my_flow():
    result = add_one(1) # return int

In [4]:
@flow 
def my_add_one_flow():
    state = add_one(1, return_state=True) # return State
    return state

In [5]:
state = my_add_one_flow()

17:25:33.625 | INFO    | prefect.engine - Created flow run 'gabby-slug' for flow 'my-add-one-flow'
17:25:33.625 | INFO    | Flow run 'gabby-slug' - Starting 'ConcurrentTaskRunner'; submitted tasks will be run concurrently...
17:25:33.699 | INFO    | Flow run 'gabby-slug' - Created task run 'add_one-3c3112ef-0' for task 'add_one'
17:25:33.700 | INFO    | Flow run 'gabby-slug' - Executing 'add_one-3c3112ef-0' immediately...
17:25:33.725 | INFO    | Task run 'add_one-3c3112ef-0' - Finished in state Completed()
17:25:33.738 | INFO    | Flow run 'gabby-slug' - Finished in state Completed('All states completed.')


In [6]:
state.result()

2

In [7]:
@flow
def my_favorite_function():
    print("This function doesn't do much")
    return 42

@flow
def my_favorite_function_flow():
    return my_favorite_function(return_state=True)

Running a Prefect workflow manually is as easy as calling the annotated function. In this case, we run the      `my_favorite_function()` snippet shown above:

In [8]:
state = my_favorite_function_flow()

17:25:33.969 | INFO    | prefect.engine - Created flow run 'idealistic-mouse' for flow 'my-favorite-function-flow'
17:25:33.969 | INFO    | Flow run 'idealistic-mouse' - Starting 'ConcurrentTaskRunner'; submitted tasks will be run concurrently...
17:25:34.050 | INFO    | Flow run 'idealistic-mouse' - Created subflow run 'conscious-salmon' for flow 'my-favorite-function'
17:25:34.098 | INFO    | Flow run 'conscious-salmon' - Finished in state Completed()
17:25:34.111 | INFO    | Flow run 'idealistic-mouse' - Finished in state Completed('All states completed.')


This function doesn't do much


The first thing you'll notice is the messages surrounding the expected output, "This function doesn't do much".

By adding the `@flow` decorator to your function, function calls will create a flow run — the Prefect Orion orchestration engine manages task and flow state, including inspecting their progress, regardless of where your flow code runs.

For clarity in future tutorial examples, we may not show these messages in results except where they are relevant to the discussion.

Next, examine `state`, which shows the result returned by the `my_favorite_function()` flow.

In [9]:
state

Completed(message=None, type=COMPLETED, result=42)

*Flows return states*

> You may notice that this call did not return the number 42 but rather a Prefect State object. States are the basic currency of communication between Prefect clients and the Prefect API, and can be used to define the conditions for orchestration rules as well as an interface for client-side logic.

In this case, the state of `my_favorite_function()` is "Completed", with no further message details ("None" in this example).

If you want to see the data returned by the flow, access it via the `.result()` method on the State object.

In [10]:
state.result()

42

### Run flows with parameters

As with any Python function, you can pass arguments. The positional and keyword arguments defined on your flow function are called parameters. To demonstrate:

In [11]:
@task
def call_api(url):
    return requests.get(url).json()


In [12]:
@flow
def api_flow(url):
    return call_api(url, return_state=True)

You can pass any parameters needed by your flow function, and you can pass parameters on the `@flow` decorator for configuration as well. We'll cover that in a future tutorial.

For now, run the `call_api()` flow, passing a valid URL as a parameter. In this case, we're sending a POST request to an API that should return valid JSON in the response.

In [13]:
state = call_api_flow("http://time.jsontest.com/")

17:25:34.439 | INFO    | prefect.engine - Created flow run 'agate-wasp' for flow 'call-api-flow'
17:25:34.439 | INFO    | Flow run 'agate-wasp' - Starting 'ConcurrentTaskRunner'; submitted tasks will be run concurrently...
17:25:34.501 | INFO    | Flow run 'agate-wasp' - Created task run 'call_api-ded10bed-0' for task 'call_api'
17:25:34.501 | INFO    | Flow run 'agate-wasp' - Executing 'call_api-ded10bed-0' immediately...
17:25:34.876 | INFO    | Task run 'call_api-ded10bed-0' - Finished in state Completed()
17:25:34.893 | INFO    | Flow run 'agate-wasp' - Finished in state Completed('All states completed.')


Again, Prefect Orion automatically orchestrates the flow run. Again, print the state and note the "Completed" state matches what Prefect Orion prints in your terminal.

In [14]:
state.result()

{'date': '07-28-2022',
 'milliseconds_since_epoch': 1658993134691,
 'time': '07:25:34 AM'}

In [15]:
state

Completed(message=None, type=COMPLETED, result={'date': '07-28-2022', 'milliseconds_since_epoch': 1658993134691, 'time': '07:25:34 AM'})

### Error handling

What happens if the Python function encounters an error while your flow is running? To see what happens whenever our flow does not complete successfully, let's intentionally run the `call_api()` flow above with a bad value for the URL:

In [17]:
# state_bad = call_api_flow("foo") - exception doesn't get caught??

In this situation, the call to `requests.get()` encounters an exception, but the flow run still returns! The exception is captured by Orion, which continues to shut down the flow run normally.

However, in contrast to the 'COMPLETED' state, we now encounter a 'FAILED' state signaling that something unexpected happened during execution.

In [18]:
state

Completed(message=None, type=COMPLETED, result={'date': '07-28-2022', 'milliseconds_since_epoch': 1658993134691, 'time': '07:25:34 AM'})

This behavior is consistent across flow runs and task runs and allows you to respond to failures in a first-class way — whether by configuring orchestration rules in the Orion backend (retry logic) or by directly responding to failed states in client code.

### Run a basic flow with tasks

Let's now add some tasks to a flow so that we can orchestrate and monitor at a more granular level.

A task is a function that represents a distinct piece of work executed within a flow. You don't have to use tasks — you can include all of the logic of your workflow within the flow itself. However, encapsulating your business logic into smaller task units gives you more granular observability, control over how specific tasks are run (potentially taking advantage of parallel execution), and reusing tasks across flows and sub-flows.

Creating and adding tasks follows the exact same pattern as for flows. Import task and use the `@task` decorator to annotate functions as tasks.

Let's take the previous `call_api()` example and move the actual HTTP request to its own task.

In [19]:
@task
def call_api(url):
    response = requests.get(url)
    print(f"Response: {response.status_code}")
    return response.json()


@flow
def api_flow(url):
    fact_json = call_api(url)
    return 

/Users/mjboothaus/code/github/mjboothaus/try-prefect2/.venv_dev_try-prefect2/lib/python3.9/site-packages/prefect/tasks.py:166: UserWarning: A task named 'call_api' and defined at '/var/folders/wc/zhl433ms1fx5rvtvcrkw6dgw0000gn/T/ipykernel_23767/3105438947.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


As you can see, we still call these tasks as normal functions and can pass their return values to other tasks. We can then call our flow function — now called `api_flow()` — just as before and see the printed output. Prefect manages all the relevant intermediate states.

In [20]:
state = api_flow("https://catfact.ninja/fact")

18:03:31.224 | INFO    | prefect.engine - Created flow run 'rational-hummingbird' for flow 'api-flow'
18:03:31.224 | INFO    | Flow run 'rational-hummingbird' - Starting 'ConcurrentTaskRunner'; submitted tasks will be run concurrently...
18:03:31.362 | INFO    | Flow run 'rational-hummingbird' - Created task run 'call_api-ded10bed-0' for task 'call_api'
18:03:31.362 | INFO    | Flow run 'rational-hummingbird' - Executing 'call_api-ded10bed-0' immediately...
18:03:32.728 | INFO    | Task run 'call_api-ded10bed-0' - Finished in state Completed()
18:03:32.752 | INFO    | Flow run 'rational-hummingbird' - Finished in state Completed('All states completed.')


Response: 200


And of course we can create tasks that take input from and pass input to other tasks.



In [21]:
@task(name="t-example3-1")
def call_api(url):
    response = requests.get(url)
    print(response.status_code)
    return response.json()

@task(name="t-example3-2")
def parse_fact(response):
    print(response["fact"])
    return 

@flow(name="f-example3")
def api_flow(url):
    fact_json = call_api(url)
    parse_fact(fact_json)
    return

In [25]:
state = api_flow("https://catfact.ninja/fact")

18:04:35.667 | INFO    | prefect.engine - Created flow run 'loose-mongrel' for flow 'f-example3'
18:04:35.668 | INFO    | Flow run 'loose-mongrel' - Starting 'ConcurrentTaskRunner'; submitted tasks will be run concurrently...
18:04:35.752 | INFO    | Flow run 'loose-mongrel' - Created task run 't-example3-1-ded10bed-0' for task 't-example3-1'
18:04:35.753 | INFO    | Flow run 'loose-mongrel' - Executing 't-example3-1-ded10bed-0' immediately...
18:04:36.479 | INFO    | Task run 't-example3-1-ded10bed-0' - Finished in state Completed()
18:04:36.498 | INFO    | Flow run 'loose-mongrel' - Created task run 't-example3-2-6803447a-0' for task 't-example3-2'
18:04:36.499 | INFO    | Flow run 'loose-mongrel' - Executing 't-example3-2-6803447a-0' immediately...
18:04:36.530 | INFO    | Task run 't-example3-2-6803447a-0' - Finished in state Completed()
18:04:36.548 | INFO    | Flow run 'loose-mongrel' - Finished in state Completed('All states completed.')


200
Cats only use their meows to talk to humans, not each other. The only time they meow to communicate with other felines is when they are kittens to signal to their mother.


In [23]:
print(state)

[Completed(message=None, type=COMPLETED, result={'fact': "Cats must have fat in their diet because they can't produce it on their own.", 'length': 76}), Completed(message=None, type=COMPLETED, result=None)]


Combining tasks with arbitrary Python code

> Notice in the above example that all of our Python logic is encapsulated within task functions. While there are many benefits to using Prefect in this way, it is not a strict requirement. Interacting with the results of your Prefect tasks requires an understanding of Prefect futures.

### Run a flow within a flow

Not only can you call tasks functions within a flow, but you can also call other flow functions! Flows that run within other flows are called sub-flows and allow you to efficiently manage, track, and version common multi-task logic.

Consider the following simple example:

In [26]:
@flow
def common_flow(config: dict):
    print("I am a subgraph that shows up in lots of places!")
    intermediate_result = 42
    return intermediate_result

@flow
def main_flow():
    # do some things
    # then call another flow function
    data = common_flow(config={})
    # do more things

Whenever we run `main_flow` as above, a new run will be generated for `common_flow` as well. Not only is this run tracked as a sub-flow run of `main_flow`, but you can also inspect it independently in the UI!

In [27]:
flow_state = main_flow()

18:04:46.628 | INFO    | prefect.engine - Created flow run 'lavender-petrel' for flow 'main-flow'
18:04:46.629 | INFO    | Flow run 'lavender-petrel' - Starting 'ConcurrentTaskRunner'; submitted tasks will be run concurrently...
18:04:46.759 | INFO    | Flow run 'lavender-petrel' - Created subflow run 'unselfish-sloth' for flow 'common-flow'
18:04:46.829 | INFO    | Flow run 'unselfish-sloth' - Finished in state Completed()
18:04:46.845 | INFO    | Flow run 'lavender-petrel' - Finished in state Completed('All states completed.')


I am a subgraph that shows up in lots of places!


You can confirm this for yourself by spinning up the UI using the `prefect orion start` CLI command from your terminal:


In [28]:
!prefect start

Usage: prefect [OPTIONS] COMMAND [ARGS]...
Try 'prefect --help' for help.
╭─ Error ──────────────────────────────────────────────────────────────────────╮
│ No such command 'start'.                                                     │
╰──────────────────────────────────────────────────────────────────────────────╯


### Asynchronous functions

Even asynchronous functions work with Prefect! Here's a variation of the previous examples that makes the API request as an async operation:

In [ ]:
@task
async def call_api(url):
    response = requests.get(url)
    print(response.status_code)
    return response.json()

@flow
async def async_flow(url):
    fact_json = await call_api(url)
    return 

If we run this in the REPL, the output looks just like previous runs.

In [ ]:
# Following doesn't seem to run in Jupyter

# asyncio.run(async_flow("https://catfact.ninja/fact"))